In [ ]:
import pandas as pd
import requests
import time
from dotenv import dotenv_values

In [ ]:
config = dotenv_values(".env")

In [ ]:
API_DOMAIN = config["DOMAIN"]
# api_abstract = config["CONTENT_DOI_URL"]
API_ABSTRACT_BASE_URL = config["CONTENT_EID_URL"]
API_SERIAL_BASE_URL = config["CONTENT_SERIAL_URL"]

API_KEY = config["API_KEY"]
TARGET_AFFILIATION_ID = config["AFFILIATION_ID"]

CONTENT_SERIAL_URL + 20961758  
header  
X-ELS-APIKey = API_KEY  
Accept = application/json  

CONTENT_DOI_URL + 10.1016/j.aaf.2024.06.003  
header  
X-ELS-APIKey = API_KEY   
Accept = application/json  

https://www.scimagojr.com/journalrank.php = scimagojr download csv

In [3]:
# df_all_data = pd.read_csv("../data/create/all_data.csv", delimiter=",")
name_file = "2005_AFID_60072061.csv"
name_file = "2006_AFID_60072061.csv"
name_file = "2007_AFID_60072061.csv"
name_file = "2008_AFID_60072061.csv"
name_file = "2009_AFID_60072061.csv"
name_file = "2010_AFID_60072061.csv"
name_file = "2011_AFID_60072061.csv"
name_file = "2024_AFID_60072061.csv"
name_file = "2023_AFID_60072061.csv"
name_file = "2022_AFID_60072061.csv"
name_file = "2021_AFID_60072061.csv"
name_file = "2020_AFID_60072061.csv"
name_file = "2019_AFID_60072061.csv"
name_file = "2018_AFID_60072061.csv"
name_file = "2017_AFID_60072061.csv"
name_file = "2016_AFID_60072061.csv"
name_file = "2015_AFID_60072061.csv"
name_file = "2014_AFID_60072061.csv"
name_file = "2012_AFID_60072061.csv"
name_file = "2025_AFID_60072061.csv"
name_file = "again.csv"

df_all_data = pd.read_csv("../data/create/" + name_file, delimiter=",")

df_scimagojr = pd.read_csv("../data/scimagojr 2024.csv", delimiter=";")

In [4]:
df_scimagojr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31136 entries, 0 to 31135
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Rank                       31136 non-null  int64 
 1   Sourceid                   31136 non-null  int64 
 2   Title                      31136 non-null  object
 3   Type                       31136 non-null  object
 4   Issn                       31136 non-null  object
 5   SJR                        30833 non-null  object
 6   SJR Best Quartile          31136 non-null  object
 7   H index                    31136 non-null  int64 
 8   Total Docs. (2024)         31136 non-null  int64 
 9   Total Docs. (3years)       31136 non-null  int64 
 10  Total Refs.                31136 non-null  int64 
 11  Total Citations (3years)   31136 non-null  int64 
 12  Citable Docs. (3years)     31136 non-null  int64 
 13  Citations / Doc. (2years)  31136 non-null  object
 14  Ref. /

In [5]:
df_all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   eid                 59 non-null     object 
 1   title               59 non-null     object 
 2   identifier          59 non-null     object 
 3   url                 59 non-null     object 
 4   urlPage             59 non-null     object 
 5   doi                 54 non-null     object 
 6   aggregationType     59 non-null     object 
 7   subtypeDescription  59 non-null     object 
 8   affiliation         59 non-null     object 
 9   publicationName     59 non-null     object 
 10  coverDate           59 non-null     object 
 11  sourceId            59 non-null     float64
 12  openaccess          59 non-null     int64  
 13  eIssn/issn/isbn     59 non-null     object 
 14  año                 59 non-null     int64  
dtypes: float64(1), int64(2), object(12)
memory usage: 7.0+ KB


In [ ]:
def get_dict_from_maybe_list(data_input):
    """
    Si data_input es una lista, devuelve el primer elemento si es un dict, sino un dict vacío.
    Si data_input es un dict, lo devuelve.
    Sino, devuelve un dict vacío.
    """
    if isinstance(data_input, list):
        if data_input and isinstance(data_input[0], dict):
            return data_input[0]
        return {}
    elif isinstance(data_input, dict):
        return data_input
    return {}

In [7]:
def format_author_name(author_data):
    if 'ce:surname' in author_data and "ce:given-name" in author_data:
        return f"{author_data.get('ce:given-name', '')} {author_data.get('ce:surname', '')}".strip()
    elif 'ce:surname' in author_data and "ce:initials" in author_data:
        return f"{author_data.get('ce:initials', '')} {author_data.get('ce:surname', '')}".strip()
    elif 'ce:indexed-name' in author_data:
        return author_data.get('ce:indexed-name', '').strip()
    return ""

In [8]:
def extract_authors_by_affiliation(response_dic, target_afid):
    authors_list = []
    affiliation_found_id = ""

    # Acceso robusto a 'author-group'
    item_data = get_dict_from_maybe_list(response_dic.get("item"))
    bibrecord_data = get_dict_from_maybe_list(item_data.get("bibrecord"))
    head_data = get_dict_from_maybe_list(bibrecord_data.get("head"))

    author_groups_data = head_data.get("author-group", []) # El default [] es importante aquí

    if not isinstance(author_groups_data, list):
        author_groups_data = [author_groups_data] if author_groups_data else []

    for group in author_groups_data:
        # Asegurarse que group es un dict antes de llamar .get
        if not isinstance(group, dict): continue
        affiliation = group.get("affiliation", {})
        if affiliation.get("@afid") == target_afid:
            affiliation_found_id = target_afid
            authors_in_group = group.get("author", [])
            if not isinstance(authors_in_group, list):
                authors_in_group = [authors_in_group] if authors_in_group else []

            for author_data in authors_in_group:
                if not isinstance(author_data, dict): continue
                name = format_author_name(author_data)
                if name:
                    authors_list.append(name)
    return ", ".join(authors_list), affiliation_found_id


In [9]:
def extract_keywords(response_dic):
    # Acceso robusto a 'authkeywords'
    authkeywords_container = get_dict_from_maybe_list(response_dic.get("authkeywords"))

    if not authkeywords_container: # Si es un dict vacío después de get_dict_from_maybe_list
        return ""

    keywords_list_data = authkeywords_container.get("author-keyword", [])
    if not isinstance(keywords_list_data, list):
        keywords_list_data = [keywords_list_data] if keywords_list_data else []

    return ", ".join(kw.get('$', "") for kw in keywords_list_data if isinstance(kw, dict) and kw.get('$'))


In [ ]:
def extract_isbn(coredata):
    isbn_list_data = coredata.get("prism:isbn", [])
    if isinstance(isbn_list_data, dict):
        isbn_list_data = [isbn_list_data]
    elif not isinstance(isbn_list_data, list):
        isbn_list_data = []
    return ", ".join(i.get("$", "") for i in isbn_list_data if isinstance(i, dict) and i.get("$"))

In [ ]:
def extract_publication_details(coredata):
    details = {
        "citations_scopus_count": coredata.get("citedby-count", ""),
        "coverDate": "", "publication_year": "", "page_raw": "", "page_count": ""
    }
    cover_date_str = coredata.get("prism:coverDate", "")
    details["coverDate"] = cover_date_str
    if cover_date_str and "-" in cover_date_str:
        details["publication_year"] = cover_date_str.split("-")[0]
    page_range_str = coredata.get("prism:pageRange", "")
    details["page_raw"] = page_range_str
    if page_range_str and "-" in page_range_str:
        try:
            parts = page_range_str.split("-")
            if len(parts) == 2:
                start_page, end_page = map(int, parts)
                details["page_count"] = end_page - start_page
            else:
                print(f"Advertencia: Formato de rango de página inesperado '{page_range_str}'.")
                details["page_count"] = ""
        except ValueError:
            print(f"Advertencia: No se pudo analizar el rango de páginas '{page_range_str}'.")
            details["page_count"] = ""
    return details

In [ ]:
def fetch_journal_details(issn_str, api_serial_url, api_key, df_scimagojr_local, eid_for_logging=""):
    journal_data = {
        "issn_processed": "", "magazine": "", "citeScore": "",
        "scimago_factor_SJR": "", "scimago_SJR_Best_Quartile": ""
    }
    if not issn_str: return journal_data
    processed_issn = issn_str.split(" ")[0]
    journal_data["issn_processed"] = processed_issn
    if len(issn_str.split(" ")) > 1:
        print(f"- Advertencia: Múltiples ISSNs ('{issn_str}') para EID {eid_for_logging}. Usando '{processed_issn}'.")
    try:
        response_issn = requests.get(
            f"{api_serial_url}{processed_issn}",
            headers={"X-ELS-APIKey": api_key, "Accept": "application/json"}
        )
        response_issn.raise_for_status()
        data_issn = response_issn.json()
        entry_container = get_dict_from_maybe_list(data_issn.get("serial-metadata-response"))
        entry = get_dict_from_maybe_list(entry_container.get("entry")) # entry puede ser lista o dict

        if entry: # Si entry no es un dict vacío
            journal_data["magazine"] = entry.get("dc:title", "")
            cite_score_info = get_dict_from_maybe_list(entry.get("citeScoreYearInfoList"))
            journal_data["citeScore"] = cite_score_info.get("citeScoreCurrentMetric", "")
        else:
            print(f"Advertencia: No se encontró entrada en serial-metadata-response para ISSN {processed_issn} (EID: {eid_for_logging}).")
    except requests.exceptions.RequestException as e:
        print(f"Error obteniendo datos seriales para ISSN {processed_issn} (EID: {eid_for_logging}): {e}")
    except ValueError:
        print(f"Error decodificando JSON para ISSN {processed_issn} (EID: {eid_for_logging}).")

    if df_scimagojr_local is not None and not df_scimagojr_local.empty:
        search = df_scimagojr_local[df_scimagojr_local["Issn"].astype(str).str.contains(processed_issn, na=False)]
        if not search.empty:
            if len(search) > 1:
                print(f"- Advertencia: Múltiples entradas Scimago para EID {eid_for_logging} ISSN {processed_issn}.")
            journal_data["scimago_factor_SJR"] = search["SJR"].iloc[0]
            journal_data["scimago_SJR_Best_Quartile"] = search["SJR Best Quartile"].iloc[0]
        else:
            print(f"- Info: No se encontró entrada Scimago para EID {eid_for_logging} ISSN {processed_issn}.")
    else:
         print(f"- Info: df_scimagojr_local no disponible. Omitiendo búsqueda Scimago para EID {eid_for_logging}.")
    return journal_data

In [13]:
all_results_list = []

In [14]:
expected_columns = [
    "url", "doi", "url_doi", "eid", "title", "abstract", "key_words",
    "cuartil", "volume", "authors", "affiliations", "issn", "magazine",
    "citeScore", "citations_scopus_count", "language", "coverDate",
    "publication_year", "page_raw", "page_count", "scimago_factor_SJR",
    "scimago_SJR_Best_Quartile", "eid_original_input", "error_message" # Columnas adicionales para trazabilidad
]

In [15]:
print(f"Procesando {len(df_all_data)} registros...")
for index, row in df_all_data.iterrows():
    eid_from_row = row["eid"]
    url_from_row = row.get("urlPage", "")
    current_result = {col: "" for col in expected_columns}
    current_result["eid_original_input"] = eid_from_row
    current_result["url"] = url_from_row
    print(f"\nProcesando registro {index + 1}/{len(df_all_data)}: EID = '{eid_from_row}'")
    try:
        response = requests.get(
            API_ABSTRACT_BASE_URL + eid_from_row,
            headers={"X-ELS-APIKey": API_KEY, "Accept": "application/json"}
        )
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP para EID '{eid_from_row}': {http_err}.")
        current_result["error_message"] = f"HTTPError: {http_err} (Code: {response.status_code if 'response' in locals() else 'N/A'})"
        if 'response' in locals() and (response.status_code == 401 or response.status_code == 403):
            print("Error API Key. Deteniendo.")
            all_results_list.append(current_result)
            break
        all_results_list.append(current_result)
        time.sleep(1)
        continue
    except requests.exceptions.RequestException as req_err:
        print(f"Error de conexión para EID '{eid_from_row}': {req_err}")
        current_result["error_message"] = f"RequestException: {req_err}"
        all_results_list.append(current_result)
        time.sleep(1)
        continue
    except ValueError:
        print(f"Error decodificando JSON para EID '{eid_from_row}'.")
        current_result["error_message"] = "JSONDecodeError"
        all_results_list.append(current_result)
        time.sleep(1)
        continue

    response_dic = data.get('abstracts-retrieval-response')
    if not response_dic: # response_dic podría ser None
        print(f"Error: 'abstracts-retrieval-response' no encontrado para EID {eid_from_row}.")
        current_result["error_message"] = "abstracts-retrieval-response missing"
        all_results_list.append(current_result)
        time.sleep(1)
        continue

    # Asegurarse que response_dic es un diccionario para el procesamiento posterior
    response_dic = get_dict_from_maybe_list(response_dic)
    if not response_dic: # Si después de get_dict_from_maybe_list sigue vacío
        print(f"Error: 'abstracts-retrieval-response' es inválido para EID {eid_from_row}.")
        current_result["error_message"] = "abstracts-retrieval-response invalid structure"
        all_results_list.append(current_result)
        time.sleep(1)
        continue


    coredata = get_dict_from_maybe_list(response_dic.get("coredata"))

    current_result["doi"] = coredata.get("prism:doi", "")
    current_result["url_doi"] = f"https://doi.org/{current_result['doi']}" if current_result['doi'] else ""
    current_result["eid"] = coredata.get("eid", eid_from_row)
    current_result["title"] = coredata.get("dc:title", "")
    current_result["abstract"] = coredata.get("dc:description", "")

    # --- Corrección para la extracción del lenguaje ---
    lang_item_data = get_dict_from_maybe_list(response_dic.get("item"))
    lang_bibrecord_data = get_dict_from_maybe_list(lang_item_data.get("bibrecord"))
    lang_head_data = get_dict_from_maybe_list(lang_bibrecord_data.get("head"))
    lang_citation_info_data = get_dict_from_maybe_list(lang_head_data.get("citation-info"))
    lang_abstract_language_data = get_dict_from_maybe_list(lang_citation_info_data.get("abstract-language"))
    current_result["language"] = lang_abstract_language_data.get("@language", "")
    # --- Fin de la corrección ---

    if not current_result["abstract"]: print(f"Info: Abstract no encontrado para EID {current_result['eid']}.")

    current_result["key_words"] = extract_keywords(response_dic) # Ya usa get_dict_from_maybe_list internamente
    current_result["volume"] = coredata.get("prism:volume", "")

    authors_str, affiliation_id_found = extract_authors_by_affiliation(response_dic, TARGET_AFFILIATION_ID) # Ya usa get_dict_from_maybe_list internamente
    current_result["authors"] = authors_str
    current_result["affiliations"] = affiliation_id_found

    pub_details = extract_publication_details(coredata)
    current_result.update(pub_details)

    issn_from_core = coredata.get("prism:issn", "")
    journal_info = {}
    final_issn_or_isbn_for_df = ""
    if issn_from_core:
        journal_info = fetch_journal_details(issn_from_core, API_SERIAL_BASE_URL, API_KEY, df_scimagojr, current_result["eid"])
        final_issn_or_isbn_for_df = journal_info.get("issn_processed", issn_from_core.split(" ")[0])
    else:
        isbn_value = extract_isbn(coredata)
        if isbn_value:
            final_issn_or_isbn_for_df = isbn_value
            print(f"Info: ISSN no encontrado para EID {current_result['eid']}. ISBN: {isbn_value}")
        else:
            print(f"Error: Ni ISSN ni ISBN encontrados para EID {current_result['eid']}.")
        journal_info = {"magazine": "", "citeScore": "", "scimago_factor_SJR": "", "scimago_SJR_Best_Quartile": ""}
    current_result["issn"] = final_issn_or_isbn_for_df
    current_result.update({k: v for k, v in journal_info.items() if k != "issn_processed"})
    current_result["cuartil"] = ""
    all_results_list.append(current_result)
    print(f"EID = '{current_result['eid']}' procesado.")
    print("*" * 100)
    time.sleep(1)


Procesando 59 registros...



Procesando registro 1/59: EID = '2-s2.0-85038931282'
EID = '2-s2.0-85038931282' procesado.
****************************************************************************************************

Procesando registro 2/59: EID = '2-s2.0-85038887707'
EID = '2-s2.0-85038887707' procesado.
****************************************************************************************************

Procesando registro 3/59: EID = '2-s2.0-85042761930'
Error: Ni ISSN ni ISBN encontrados para EID 2-s2.0-85042761930.
EID = '2-s2.0-85042761930' procesado.
****************************************************************************************************

Procesando registro 4/59: EID = '2-s2.0-85046476921'
Error: Ni ISSN ni ISBN encontrados para EID 2-s2.0-85046476921.
EID = '2-s2.0-85046476921' procesado.
****************************************************************************************************

Procesando registro 5/59: EID = '2-s2.0-85046415213'
Error: Ni ISSN ni ISBN encontrados para EID 2

In [16]:
df_resultados_final = pd.DataFrame(all_results_list, columns=expected_columns)

In [17]:
df_resultados_final.shape

(59, 24)

In [18]:
df_resultados_final.head()

,url,doi,url_doi,eid,title,abstract,key_words,cuartil,volume,authors,...,citations_scopus_count,language,coverDate,publication_year,page_raw,page_count,scimago_factor_SJR,scimago_SJR_Best_Quartile,eid_original_input,error_message
0,https://www.scopus.com/record/display.uri?eid=...,10.1186/s13018-017-0700-2,https://doi.org/10.1186/s13018-017-0700-2,2-s2.0-85038931282,Biomechanical properties of 3D-printed bone sc...,Background: One of the major challenges in ort...,,,12,Carlos G. Helguero,...,20,English,2017-12-22,2017,,,"0,877",Q1,2-s2.0-85038931282,
1,https://www.scopus.com/record/display.uri?eid=...,10.1186/s13104-017-3083-x,https://doi.org/10.1186/s13104-017-3083-x,2-s2.0-85038887707,Comparison of three DNA extraction methods for...,"Objectives: In Ecuador, food products need to ...","DNA extraction, GMOs, PCR, Processed food",,10,"Ricardo Pacheco Coello, Efrén Santos Ordoñez",...,25,English,2017-12-20,2017,,,"0,435",Q3,2-s2.0-85038887707,
2,https://www.scopus.com/record/display.uri?eid=...,10.1109/CHILECON.2017.8229687,https://doi.org/10.1109/CHILECON.2017.8229687,2-s2.0-85042761930,On the influence of spectral calibration in hy...,Automatic detection of physiological changes i...,"Hyperspectral imaging, Spectral calibration, S...",,2017-January,"Rodrigo Castro, Daniel Ochoa, Ronald Criollo",...,0,English,2017-12-19,2017,1-6,5,,,2-s2.0-85042761930,
3,https://www.scopus.com/record/display.uri?eid=...,10.1109/CLEI.2017.8226423,https://doi.org/10.1109/CLEI.2017.8226423,2-s2.0-85046476921,Multi-objective maximum diversity problem,The Maximum Diversity (MD) problem is the proc...,"Multi-Objective Evolutionary Algorithm, Multi-...",,2017-January,Fernando Sandoya,...,5,English,2017-12-18,2017,1-9,8,,,2-s2.0-85046476921,
4,https://www.scopus.com/record/display.uri?eid=...,10.1109/CLEI.2017.8226454,https://doi.org/10.1109/CLEI.2017.8226454,2-s2.0-85046415213,A multi-objective two-echelon vehicle routing ...,Current trends of urbanization and growing eco...,"City Distribution Centers, Multi-Objective Opt...",,2017-January,Fernando Sandoya,...,20,English,2017-12-18,2017,1-10,9,,,2-s2.0-85046415213,


In [19]:
df_resultados_final.to_csv("../data/process/" + name_file)

In [22]:
df_resultados_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   url                        59 non-null     object
 1   doi                        59 non-null     object
 2   url_doi                    59 non-null     object
 3   eid                        59 non-null     object
 4   title                      59 non-null     object
 5   abstract                   59 non-null     object
 6   key_words                  59 non-null     object
 7   cuartil                    59 non-null     object
 8   volume                     59 non-null     object
 9   authors                    59 non-null     object
 10  affiliations               59 non-null     object
 11  issn                       59 non-null     object
 12  magazine                   59 non-null     object
 13  citeScore                  55 non-null     object
 14  citations_sc